# Kategoryzacja zmiennych

Zbiór danych do analizy: https://www.kaggle.com/datasets/clemencetravers/predict-mc-donalds-stock-price

Mamy dane dotyczące wskaźników giełdowych, na podstawie których mamy zaprognozować, czy cena akcji wzrośnie czy zmaleje.

Zmienne:
- S&P500
- Dow Jones
- Wendy's Index
- Yum's index (Taco Bell, Pizza Hut etc.)
- Starbuck's index
- Coca's index
- Wheat index: Chicago SRW Wheat Future (ZW=F)
- Oil index: Crude oil (CL=F)
- Commodity: United State Commodity Index
- Sugar: (SB=F)
- Volatility: VXD index
- War: War in Ukraine 0 : no war, 1: war (the 20/02/2022)

In [ ]:
#pip install scorecardpy

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import SplineTransformer, KBinsDiscretizer
import scorecardpy as sc
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [10]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [11]:
# Pobranie danych
df = pd.read_csv("data/stock_price.csv",sep=';',decimal=',')

In [ ]:
# Nagłówek
df.head()

In [ ]:
# Liczebność zmiennej celu
df['MCD'].value_counts()

In [ ]:
df.info()

## Podział według kwantyli

In [7]:
# Stworzenie obiektu - one hot
oil_quantile = KBinsDiscretizer(strategy='quantile',n_bins=10).fit_transform(df[['Oil']])

In [ ]:
# wyniki
oil_quantile.toarray()

In [ ]:
oil_quantile.toarray()[0]

In [11]:
# Stworzenie obiektu - ordinal
oil_quantile_ordinal = KBinsDiscretizer(strategy='quantile',n_bins=10,encode = 'ordinal').fit_transform(df[['Oil']])

In [ ]:
# wyniki
oil_quantile_ordinal

In [13]:
# inaczej
df['Oil_q'] = pd.qcut(df['Oil'], q=5)

In [ ]:
df['Oil_q'].value_counts()

## Podział według równych rozpiętości

In [15]:
# stworzenie obiektu
oil_uniform = KBinsDiscretizer(strategy='uniform',encode='ordinal').fit_transform(df[['Oil']])

In [16]:
df['oil_uniform'] = oil_uniform 

In [ ]:
df['oil_uniform'].value_counts()

## WoE i IV

In [ ]:
# Sprawdzenie korelacji
df.select_dtypes(exclude = 'category').corr(method='spearman')

In [19]:
# Podział zbioru
train, test, train_y,test_y = train_test_split(df.drop('Oil_q',axis=1),df['MCD'], test_size=0.2, random_state=123)

In [ ]:
# Stworzenie podziałów zmiennych
bins = sc.woebin(dt=train,y = 'MCD')

In [ ]:
# Podział
bins

In [ ]:
# Wykres woebin
plt.rcParams['figure.figsize'] =[10,5]
sc.woebin_plot(bins)
plt.show()

In [ ]:
# Konwersja zmiennych na WoE
train_woe = sc.woebin_ply(train,bins)

In [ ]:
# head
train_woe.head()


In [26]:
# test modelu regresji logistycznej
lr = LogisticRegression().fit(train_woe.drop('MCD',axis=1),train_woe['MCD'])

In [27]:
# stworzenie score card
card = sc.scorecard(bins, lr, lr.feature_names_in_)

In [ ]:
card

In [ ]:
sc.scorecard_ply(train, card, print_step=0)